# Div Arb Straight Up

**The Goal of this notebook is to locate areas of "straight up arbitrage" where we can gather at least .5% 2-day profit on liquid options**

*We Define "Straight Up Arb" as *Arbitrage from purchasing a put close to expiry where*     
`Div*(1-tax) - Cost_option < Underlying - Strike`   
We define percentage of arbitrage as The Percentage of Arbitrage that we can get from this as   
`Div * (1 - tax) / (Underlying + Cost_Option)`   
Since we can put on a position daily we would be satisfied with anything like a .2% return (keep in mind that our profits only get compounded every 1.5 months since there is a 1.5 month delay before getting payment from the dividend. In fact we may slowly bleed liquidity until dividend payments start coming through).

In [ ]:
#Dividend*(1-tax)/(Strike+Cost)

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bisect import bisect_left

In [12]:
pd.set_option('display.max_columns', 500)
PATH_TO_DIV = "gs://ucf-data/dividends/"
PATH_TO_OPT = "gs://ucf-data/Options/"
PATH_TO_DAILY = "gs://ucf-data/day-normalized-market-data/"

In [66]:
def get_dividend_df(ticker):
    df = pd.read_csv('{}{}.csv'.format(PATH_TO_DIV,ticker.upper()))
    df['DATE'] = df.apply (lambda row: datetime.strptime(row['exOrEffDate'],"%m/%d/%Y"), axis=1)
    return df
def get_opt_df(ticker):
    opts = pd.read_csv('{}{}.csv'.format(PATH_TO_OPT,ticker.upper()))
    opts['date'] = opts['date'].apply(str)
    opts['DATE'] = opts.apply (lambda row: datetime.strptime(row['date'],"%Y%m%d"), axis=1)
    return opts
def get_daily_df(ticker):
    daily = pd.read_csv('{}{}/{}_day_close_.csv'.format(PATH_TO_DAILY, ticker.upper(), ticker.upper()))
    daily['DATE'] = daily.apply(lambda row: datetime.strptime(row['timestamp'],"%Y-%m-%d"), axis=1)
    return daily
def convert_date(s):
    a = datetime.strptime(s,"%d/%m/%Y")
    return a.strftime("%Y%m%d")
def previous_day(dates, current_day):
    prev = current_day - timedelta(days=1)
    #print(prev)
    insertion_point = bisect_left(dates, current_day)-1
    if insertion_point == -1:
        return False
    else:
        return dates[insertion_point]
def gen_dates(df):
    return sorted(datetime.strptime(str(elt), "%Y%m%d") for elt in list(df.date))

In [ ]:
df = pd.read_csv('gs://ucf-data/day-data.zip'.format(PATH_TO_DIV,ticker.upper()))

In [9]:
!gsutil ls gs://ucf-data/day-normalized-market-data/AAC/

gs://ucf-data/day-normalized-market-data/AAC/AAC_day_close_.csv


In [49]:
get_daily_df("AAPL").head()

,timestamp,open,high,low,close,volume,millis,timestamp.1,time_of_last_tick
0,2000-01-03,3.746,4.018,3.632,3.750,125216004.0,946909800.0,2000-01-03,2000-01-03
1,2000-01-04,3.866,3.951,3.614,3.866,120027600.0,946996200.0,2000-01-04,2000-01-04
2,2000-01-05,3.706,3.949,3.679,3.703,182036428.0,947082600.0,2000-01-05,2000-01-05
3,2000-01-06,3.791,3.822,3.393,3.793,181484828.0,947169000.0,2000-01-06,2000-01-06
4,2000-01-07,3.447,3.608,3.411,3.447,106187200.0,947255400.0,2000-01-07,2000-01-07


In [19]:
get_opt_df("AAPL").head()

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
0,101594,20150630,AAPL 150702C100000,1,20150702,20150630.0,C,100000,25.55,25.65,15,6,1.657665,0.971807,0.004203,0.601158,-90.97760,108201233,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
1,101594,20150630,AAPL 150702C105000,1,20150702,20150630.0,C,105000,20.55,20.65,16,7,1.352380,0.966076,0.006003,0.699296,-86.50655,108201234,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
2,101594,20150630,AAPL 150702C107000,1,20150702,20150630.0,C,107000,18.25,18.80,10,0,1.115425,0.975360,0.005581,0.536270,-54.77605,108249631,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
3,101594,20150630,AAPL 150702C108000,1,20150702,NaN,C,108000,17.05,17.80,0,0,NaN,NaN,NaN,NaN,NaN,108249632,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
4,101594,20150630,AAPL 150702C109000,1,20150702,20150630.0,C,109000,16.55,16.65,286,366,1.112591,0.959576,0.008420,0.807882,-82.13522,108051023,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0


In [23]:
get_dividend_df("AAPL").head()

,exOrEffDate,type,amount,declarationDate,recordDate,paymentDate,Unnamed: 6
0,11/07/2019,Cash,$0.77,10/30/2019,11/11/2019,11/14/2019,NaN
1,08/09/2019,Cash,$0.77,07/30/2019,08/12/2019,08/15/2019,NaN
2,05/10/2019,Cash,$0.77,04/30/2019,05/13/2019,05/16/2019,NaN
3,02/08/2019,Cash,$0.73,01/29/2019,02/11/2019,02/14/2019,NaN
4,11/08/2018,Cash,$0.73,11/01/2018,11/12/2018,11/15/2018,NaN


## Step 1: Merge 3 Tables Together (Code)

In [67]:
def merge(ticker):
    dividends = get_dividend_df(ticker)
    opts = get_opt_df(ticker)
    daily = get_daily_df(ticker)
    temp = pd.merge(dividends, daily, on='DATE', how='outer')
    last = pd.merge(opts, temp, on='DATE')
    return last

In [72]:
def merge_before(ticker, days):
    dividends = get_dividend_df(ticker)
    dividends['DATE'] = dividends.apply (lambda row: row['DATE'] - timedelta(days=days), axis=1)
    opts = get_opt_df(ticker)
    daily = get_daily_df(ticker)
    temp = pd.merge(dividends, daily, on='DATE', how='outer')
    last = pd.merge(opts, temp, on='DATE')
    return last

In [97]:
TAX = .25
def calculate_arb(row):
    #Div*(1-tax) - Cost_option < Underlying - Strike
    Div = float(row['amount'][1:])*(1-TAX)
    Cost_Option = row['best_offer']
    Call = row['cp_flag'] == 'C'
    Underlying = row['close']
    Strike = row['strike_price']/1000
    ITM = Strike > Underlying
    
    print(Div, Cost_Option, Call, Underlying, Strike)
    

    if not Call and ITM:
        return Underlying-Strike+Cost_Option-Div
    return 0

In [71]:
result = merge_day_before('GE')

In [98]:
filtered = result[result.exOrEffDate.notnull()]

In [99]:
filtered.head()

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume_x,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag,DATE,exOrEffDate,type,amount,declarationDate,recordDate,paymentDate,Unnamed: 6,timestamp,open,high,low,close,volume_y,millis,timestamp.1,time_of_last_tick
39186,105169,20150916,GE 150918C15000,1,20150918,20150916.0,C,15000,10.55,11.00,420,219,NaN,NaN,NaN,NaN,NaN,106240072,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16
39187,105169,20150916,GE 150918C16000,1,20150918,NaN,C,16000,8.45,10.20,0,0,NaN,NaN,NaN,NaN,NaN,107284360,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16
39188,105169,20150916,GE 150918C17000,1,20150918,20150617.0,C,17000,7.80,9.15,0,0,NaN,NaN,NaN,NaN,NaN,107284361,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16
39189,105169,20150916,GE 150918C17500,1,20150918,NaN,C,17500,7.10,8.60,0,0,NaN,NaN,NaN,NaN,NaN,108806256,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16
39190,105169,20150916,GE 150918C18000,1,20150918,20150916.0,C,18000,7.60,8.05,420,189,NaN,NaN,NaN,NaN,NaN,106240073,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16


In [100]:
filtered['ARB'] = filtered.apply(lambda row: calculate_arb(row), axis=1)

0.17250000000000001 11.0 True 25.31 15.0
0.17250000000000001 10.2 True 25.31 16.0
0.17250000000000001 9.15 True 25.31 17.0
0.17250000000000001 8.6 True 25.31 17.5
0.17250000000000001 8.05 True 25.31 18.0
0.17250000000000001 7.6 True 25.31 18.5
0.17250000000000001 7.05 True 25.31 19.0
0.17250000000000001 6.6 True 25.31 19.5
0.17250000000000001 6.05 True 25.31 20.0
0.17250000000000001 5.55 True 25.31 20.5
0.17250000000000001 5.05 True 25.31 21.0
0.17250000000000001 4.55 True 25.31 21.5
0.17250000000000001 4.05 True 25.31 22.0
0.17250000000000001 3.5 True 25.31 22.5
0.17250000000000001 2.99 True 25.31 23.0
0.17250000000000001 2.5 True 25.31 23.5
0.17250000000000001 1.94 True 25.31 24.0
0.17250000000000001 1.5 True 25.31 24.5
0.17250000000000001 0.95 True 25.31 25.0
0.17250000000000001 0.45 True 25.31 25.5
0.17250000000000001 0.13 True 25.31 26.0
0.17250000000000001 0.03 True 25.31 26.5
0.17250000000000001 0.01 True 25.31 27.0
0.17250000000000001 0.01 True 25.31 27.5
0.17250000000000001 0.

0.17250000000000001 0.06 False 25.31 18.5
0.17250000000000001 0.07 False 25.31 19.0
0.17250000000000001 0.07 False 25.31 19.5
0.17250000000000001 0.06 False 25.31 20.0
0.17250000000000001 0.06 False 25.31 20.5
0.17250000000000001 0.13 False 25.31 21.0
0.17250000000000001 0.13 False 25.31 21.5
0.17250000000000001 0.13 False 25.31 22.0
0.17250000000000001 0.15 False 25.31 22.5
0.17250000000000001 0.19 False 25.31 23.0
0.17250000000000001 0.23 False 25.31 23.5
0.17250000000000001 0.29 False 25.31 24.0
0.17250000000000001 0.34 False 25.31 24.5
0.17250000000000001 0.46 False 25.31 25.0
0.17250000000000001 0.63 False 25.31 25.5
0.17250000000000001 0.89 False 25.31 26.0
0.17250000000000001 1.2 False 25.31 26.5
0.17250000000000001 1.54 False 25.31 27.0
0.17250000000000001 2.15 False 25.31 27.5
0.17250000000000001 2.56 False 25.31 28.0
0.17250000000000001 3.45 False 25.31 28.5
0.17250000000000001 3.95 False 25.31 29.0
0.17250000000000001 4.65 False 25.31 29.5
0.17250000000000001 5.0 False 25.31

0.17250000000000001 11.7 True 30.44 21.0
0.17250000000000001 9.15 True 30.44 22.0
0.17250000000000001 9.1 True 30.44 22.5
0.17250000000000001 9.7 True 30.44 23.0
0.17250000000000001 8.8 True 30.44 23.5
0.17250000000000001 8.3 True 30.44 24.0
0.17250000000000001 7.75 True 30.44 24.5
0.17250000000000001 6.25 True 30.44 25.0
0.17250000000000001 6.8 True 30.44 25.5
0.17250000000000001 6.0 True 30.44 26.0
0.17250000000000001 4.95 True 30.44 26.5
0.17250000000000001 4.3 True 30.44 27.0
0.17250000000000001 4.2 True 30.44 27.5
0.17250000000000001 3.3 True 30.44 28.0
0.17250000000000001 2.82 True 30.44 28.5
0.17250000000000001 2.15 True 30.44 29.0
0.17250000000000001 1.76 True 30.44 29.5
0.17250000000000001 1.05 True 30.44 30.0
0.17250000000000001 0.62 True 30.44 30.5
0.17250000000000001 0.3 True 30.44 31.0
0.17250000000000001 0.12 True 30.44 31.5
0.17250000000000001 0.06 True 30.44 32.0
0.17250000000000001 0.05 True 30.44 32.5
0.17250000000000001 0.05 True 30.44 33.0
0.17250000000000001 0.04 T

0.17250000000000001 0.38 False 30.44 28.5
0.17250000000000001 0.47 False 30.44 29.0
0.17250000000000001 0.59 False 30.44 29.5
0.17250000000000001 0.67 False 30.44 30.0
0.17250000000000001 0.84 False 30.44 30.5
0.17250000000000001 1.1 False 30.44 31.0
0.17250000000000001 1.5 False 30.44 31.5
0.17250000000000001 1.8 False 30.44 32.0
0.17250000000000001 2.13 False 30.44 32.5
0.17250000000000001 3.55 False 30.44 33.0
0.17250000000000001 4.1 False 30.44 33.5
0.17250000000000001 4.5 False 30.44 34.0
0.17250000000000001 4.95 False 30.44 34.5
0.17250000000000001 5.45 False 30.44 35.0
0.17250000000000001 6.55 False 30.44 35.5
0.17250000000000001 7.05 False 30.44 36.0
0.17250000000000001 7.55 False 30.44 36.5
0.17250000000000001 7.3 False 30.44 37.0
0.17250000000000001 8.55 False 30.44 37.5
0.17250000000000001 8.5 False 30.44 38.0
0.17250000000000001 9.7 False 30.44 38.5
0.17250000000000001 10.2 False 30.44 39.0
0.17250000000000001 10.55 False 30.44 39.5
0.17250000000000001 18.7 True 30.44 14.0


0.17250000000000001 0.03 True 29.0 39.5
0.17250000000000001 0.04 False 29.0 19.0
0.17250000000000001 0.04 False 29.0 20.0
0.17250000000000001 0.04 False 29.0 21.0
0.17250000000000001 0.04 False 29.0 21.5
0.17250000000000001 0.04 False 29.0 22.0
0.17250000000000001 0.05 False 29.0 22.5
0.17250000000000001 0.05 False 29.0 23.0
0.17250000000000001 0.06 False 29.0 23.5
0.17250000000000001 0.07 False 29.0 24.0
0.17250000000000001 0.09 False 29.0 24.5
0.17250000000000001 0.1 False 29.0 25.0
0.17250000000000001 0.07 False 29.0 25.5
0.17250000000000001 0.13 False 29.0 26.0
0.17250000000000001 0.13 False 29.0 26.5
0.17250000000000001 0.16 False 29.0 27.0
0.17250000000000001 0.21 False 29.0 27.5
0.17250000000000001 0.32 False 29.0 28.0
0.17250000000000001 0.46 False 29.0 28.5
0.17250000000000001 0.71 False 29.0 29.0
0.17250000000000001 1.03 False 29.0 29.5
0.17250000000000001 1.43 False 29.0 30.0
0.17250000000000001 2.37 False 29.0 30.5
0.17250000000000001 2.37 False 29.0 31.0
0.1725000000000000

...

0.0075 0.81 False 9.1 10.0
0.0075 1.2 False 9.1 10.5
0.0075 1.68 False 9.1 11.0
0.0075 2.16 False 9.1 11.5
0.0075 2.66 False 9.1 12.0
0.0075 3.15 False 9.1 12.5
0.0075 3.65 False 9.1 13.0
0.0075 4.35 False 9.1 13.5
0.0075 4.9 False 9.1 14.0
0.0075 5.15 False 9.1 14.5
0.0075 5.65 False 9.1 15.0
0.0075 6.15 False 9.1 15.5
0.0075 6.65 False 9.1 16.0
0.0075 0.02 False 9.1 4.0
0.0075 0.02 False 9.1 4.5
0.0075 0.02 False 9.1 5.5
0.0075 0.03 False 9.1 6.0
0.0075 0.04 False 9.1 6.5
0.0075 0.04 False 9.1 7.0
0.0075 0.1 False 9.1 7.5
0.0075 0.09 False 9.1 8.0
0.0075 0.16 False 9.1 8.5
0.0075 0.29 False 9.1 9.0
0.0075 0.51 False 9.1 9.5
0.0075 0.3 True 9.1 10.0
0.0075 0.16 True 9.1 10.5
0.0075 0.09 True 9.1 11.0
0.0075 0.05 True 9.1 11.5
0.0075 0.03 True 9.1 12.0
0.0075 0.06 True 9.1 12.5
0.0075 0.03 True 9.1 13.0
0.0075 0.04 True 9.1 13.5
0.0075 0.03 True 9.1 14.0
0.0075 0.03 True 9.1 14.5
0.0075 0.03 True 9.1 15.0
0.0075 0.03 True 9.1 15.5
0.0075 0.03 True 9.1 16.0
0.0075 4.1 True 9.1 5.5
0.007

0.0075 3.45 True 9.1 7.5
0.0075 3.1 True 9.1 8.0
0.0075 2.15 True 9.1 8.5
0.0075 1.6 True 9.1 9.0
0.0075 0.37 True 9.1 9.5
0.0075 0.3 False 9.1 10.0
0.0075 0.88 False 9.1 10.5
0.0075 1.5 False 9.1 11.0
0.0075 3.1 False 9.1 11.5
0.0075 4.0 False 9.1 12.0
0.0075 4.45 False 9.1 12.5
0.0075 4.9 False 9.1 13.0
0.0075 5.9 False 9.1 13.5
0.0075 5.95 False 9.1 14.0
0.0075 6.3 False 9.1 14.5
0.0075 7.4 False 9.1 15.0
0.0075 0.02 False 9.1 4.0
0.0075 0.02 False 9.1 4.5
0.0075 0.02 False 9.1 5.0
0.0075 0.02 False 9.1 5.5
0.0075 0.02 False 9.1 6.0
0.0075 0.01 False 9.1 6.5
0.0075 0.01 False 9.1 7.0
0.0075 0.03 False 9.1 7.5
0.0075 0.02 False 9.1 8.0
0.0075 0.01 False 9.1 8.5
0.0075 0.02 False 9.1 9.0
0.0075 0.12 False 9.1 9.5
0.0075 10.9 True 9.1 1.0
0.0075 0.36 True 9.1 10.0
0.0075 0.29 True 9.1 10.5
0.0075 0.1 True 9.1 11.0
0.0075 0.03 True 9.1 11.5
0.0075 0.03 True 9.1 12.0
0.0075 0.05 True 9.1 12.5
0.0075 0.01 True 9.1 13.0
0.0075 0.01 True 9.1 13.5
0.0075 0.01 True 9.1 14.0
0.0075 0.03 True 9

0.0075 0.03 False 9.1 2.0
0.0075 12.4 False 9.1 20.0
0.0075 0.07 False 9.1 3.0
0.0075 0.1 False 9.1 4.0
0.0075 0.18 False 9.1 5.0
0.0075 0.18 False 9.1 6.0
0.0075 0.4 False 9.1 7.0
0.0075 0.61 False 9.1 8.0
0.0075 0.97 False 9.1 9.0
0.0075 10.9 True 9.1 1.0
0.0075 1.5 True 9.1 10.0
0.0075 1.12 True 9.1 11.0
0.0075 0.65 True 9.1 12.0
0.0075 0.53 True 9.1 13.0
0.0075 0.36 True 9.1 14.0
0.0075 0.3 True 9.1 15.0
0.0075 0.1 True 9.1 16.0
0.0075 0.07 True 9.1 17.0
0.0075 0.14 True 9.1 18.0
0.0075 0.11 True 9.1 19.0
0.0075 9.9 True 9.1 2.0
0.0075 0.09 True 9.1 20.0
0.0075 8.95 True 9.1 3.0
0.0075 8.0 True 9.1 4.0
0.0075 6.5 True 9.1 5.0
0.0075 5.55 True 9.1 6.0
0.0075 4.2 True 9.1 7.0
0.0075 2.64 True 9.1 8.0
0.0075 2.02 True 9.1 9.0
0.0075 0.02 False 9.1 1.0
0.0075 1.51 False 9.1 10.0
0.0075 2.17 False 9.1 11.0
0.0075 2.89 False 9.1 12.0
0.0075 3.65 False 9.1 13.0
0.0075 5.55 False 9.1 14.0
0.0075 7.45 False 9.1 15.0
0.0075 8.4 False 9.1 16.0
0.0075 9.4 False 9.1 17.0
0.0075 10.4 False 9.1 1

0.0075 0.02 True 10.32 17.0
0.0075 0.02 True 10.32 18.0
0.0075 0.02 True 10.32 19.0
0.0075 8.4 True 10.32 2.0
0.0075 0.02 True 10.32 20.0
0.0075 7.4 True 10.32 3.0
0.0075 6.45 True 10.32 4.0
0.0075 5.45 True 10.32 5.0
0.0075 4.45 True 10.32 6.0
0.0075 3.45 True 10.32 7.0
0.0075 2.47 True 10.32 8.0
0.0075 1.56 True 10.32 9.0
0.0075 0.39 False 10.32 10.0
0.0075 0.91 False 10.32 11.0
0.0075 1.7 False 10.32 12.0
0.0075 2.64 False 10.32 13.0
0.0075 3.65 False 10.32 14.0
0.0075 4.65 False 10.32 15.0
0.0075 5.65 False 10.32 16.0
0.0075 6.65 False 10.32 17.0
0.0075 7.65 False 10.32 18.0
0.0075 8.65 False 10.32 19.0
0.0075 0.02 False 10.32 2.0
0.0075 9.65 False 10.32 20.0
0.0075 0.02 False 10.32 3.0
0.0075 0.02 False 10.32 4.0
0.0075 0.02 False 10.32 5.0
0.0075 0.02 False 10.32 6.0
0.0075 0.02 False 10.32 7.0
0.0075 0.05 False 10.32 8.0
0.0075 0.14 False 10.32 9.0
0.0075 0.95 True 10.32 10.0
0.0075 0.46 True 10.32 11.0
0.0075 0.18 True 10.32 12.0
0.0075 0.07 True 10.32 13.0
0.0075 0.04 True 10.

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
filtered[filtered['ARB'] < 0]

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume_x,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag,DATE,exOrEffDate,type,amount,declarationDate,recordDate,paymentDate,Unnamed: 6,timestamp,open,high,low,close,volume_y,millis,timestamp.1,time_of_last_tick,ARB
39249,105169,20150916,GE 150918P25500,1,20150918,20150916.0,P,25500,0.16,0.17,4927,1952,0.334119,-0.368011,0.584058,0.718930,-21.90346,108806282,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,-0.1925
39250,105169,20150916,GE 150918P26000,1,20150918,20150916.0,P,26000,0.41,0.43,1120,25605,0.315750,-0.679846,0.580159,0.675080,-19.41502,106244431,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,-0.4325
39251,105169,20150916,GE 150918P26500,1,20150918,20150916.0,P,26500,0.72,0.98,3,48,0.377140,-0.853123,0.303777,0.421776,-14.48736,108806283,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,-0.3825
39252,105169,20150916,GE 150918P27000,1,20150918,20150916.0,P,27000,1.24,1.35,256,23839,NaN,NaN,NaN,NaN,NaN,105996453,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,-0.5125
39253,105169,20150916,GE 150918P27500,1,20150918,20150831.0,P,27500,1.69,1.89,0,10,NaN,NaN,NaN,NaN,NaN,108806284,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,-0.4725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662948,105169,20190627,GE1 200117P25000,1,20200117,20190620.0,P,25000,14.15,14.30,0,0,NaN,NaN,NaN,NaN,NaN,117999754,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2019,Cash,$0.01,06/21/2019,07/01/2019,07/25/2019,NaN,2019-06-27,10.32,10.44,10.22,10.32,28629926.0,1.561627e+09,2019-06-27,2019-06-27,-0.3875
662949,105169,20190627,GE1 200117P27000,1,20200117,20190214.0,P,27000,16.15,16.30,0,0,NaN,NaN,NaN,NaN,NaN,117999755,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2019,Cash,$0.01,06/21/2019,07/01/2019,07/25/2019,NaN,2019-06-27,10.32,10.44,10.22,10.32,28629926.0,1.561627e+09,2019-06-27,2019-06-27,-0.3875
662951,105169,20190627,GE1 200117P30000,1,20200117,20190222.0,P,30000,19.10,19.30,0,0,NaN,NaN,NaN,NaN,NaN,117999757,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2019,Cash,$0.01,06/21/2019,07/01/2019,07/25/2019,NaN,2019-06-27,10.32,10.44,10.22,10.32,28629926.0,1.561627e+09,2019-06-27,2019-06-27,-0.3875
662952,105169,20190627,GE1 200117P35000,1,20200117,20190212.0,P,35000,24.10,24.30,0,0,NaN,NaN,NaN,NaN,NaN,117999758,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2

In [95]:
for elt in filtered['ARB']:
    print (elt)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.2500000000000013
-0.4900000000000013
-0.4400000000000013
-0.5700000000000012
-0.5300000000000014
-0.5300000000000011
-0.5300000000000011
-0.5200000000000014
-0.5200000000000014
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.4700000000000011
-0.5200000000000009
-0.3200000000000016
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
-0.5200000000000009
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.1200000000000013
-0.39000000000000123

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.08999999999999855
-0.11000000000000146
-0.010000000000001369
0.11999999999999852
-0.1600000000000015
-0.15000000000000127
1.2199999999999984
-0.13000000000000125
-0.13000000000000125
1.2199999999999984
1.1699999999999986
-0.13000000000000125
-0.08000000000000143
1.1699999999999986
1.2199999999999993
1.319999999999999
1.4699999999999993
1.569999999999999
1.569999999999999
1.2199999999999993
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.1899999999999986
-1.4710455076283324e-15
-0.13000000000000148
0.45999999999999863
-0.1600000000000015
-0.1700000000000013
-0.13000000000000125
-0.13000000000000125
-0.13000000

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.32000000000000073
0.34000000000000075
0.2700000000000007
0.2700000000000007
0.36000000000000054
0.8600000000000005
0.36000000000000054
1.510000000000001
0.3100000000000007
1.510000000000001
0.41000000000000036
1.6600000000000004
0.36000000000000054
1.51
0.8100000000000007
1.960000000000001
1.960000000000001
2.01
0.8600000000000014
1.9100000000000004
0.51
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3900000000000008
0.34000000000000075
0.3100000000000007
1.7100000000000006
0.8600000000000005
1.960000000000001
1.5600000000000007
1.960000000000001
1.6100000000000005
1.8100000000000007
1.8100000000000007
1.960000000000001
1.710000000000001
1.960000000000001
1.8100000000000007
2.01
1.810000000

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.25000000000000056
0.25000000000000056
0.25000000000000056
0.25000000000000056
0.25000000000000056
0.37000000000000066
0.3200000000000004
0.3200000000000004
0.3200000000000004
0.3200000000000004
0.3200000000000004
0.3200000000000004
0.3200000000000004
0.3200000000000004
0.3200000000000013
0.3200000000000013
0.3200000000000013
0.3700000000000002
0.3700000000000002
0.3200000000000013
0.3200000000000013
0.3200000000000013
0.3200000000000013
0.370000000000002
0.41999999999999915
0.370000000000002
0.3200000000000013
0.3200000000000013
0.41999999999999915
0.370000000000002
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3100000000000006
0.26000000000000056
0.25000000000000056
0.25000000000000056
0.25000000000000056
0.3200000000000004
0.3200000000000004
0.3200000000000004
0.2700000000000006
0.3200000000000004
0.3200000000000004
0.32

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.46999999999999953
0.09999999999999944
-0.2000000000000004
-0.2100000000000002
0.5899999999999996
1.5399999999999998
1.49
1.49
1.49
1.4399999999999993
0.0
1.49
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5199999999999996
0.1999999999999995
-0.08000000000000028
-0.3100000000000003
0.7899999999999998
1.74
1.6899999999999993
1.6899999999999993
1.49
1.49
0.0
1.49
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5999999999999996
0.25999999999999956
-0.020000000000000233
-0.26000000000000045
0.6399999999999995
1.5399999999999998
1.49
1.49
1.49
1.49
0.0
1.49
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7399999999999995
-0.2100000000000002
-0.2100000000000002
1.8399999999999996
0.0
0.08999999999999965
1.6899999999999993
1.7399999999999982
1.7900

In [57]:
opts = get_opt_df('GE')
opts

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
0,105169,20150630,GE 150702C18000,1,20150702,NaN,C,18000,8.50,8.75,0,0,2.819410,0.975626,0.010329,0.112796,-29.00871,107980807,1,0,100,0,26.570218,w,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
1,105169,20150630,GE 150702C19000,1,20150702,NaN,C,19000,7.35,7.85,0,0,2.241742,0.982329,0.009896,0.085733,-17.58132,107859278,1,0,100,0,26.570218,w,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
2,105169,20150630,GE 150702C20000,1,20150702,NaN,C,20000,6.50,7.10,0,0,2.956884,0.920307,0.025496,0.291730,-78.71164,107859279,1,0,100,0,26.570218,w,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
3,105169,20150630,GE 150702C20500,1,20150702,NaN,C,20500,5.80,6.25,0,0,NaN,NaN,NaN,NaN,NaN,107859280,1,0,100,0,26.570218,w,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
4,105169,20150630,GE 150702C21000,1,20150702,NaN,C,21000,5.45,5.75,0,0,1.650943,0.976528,0.017090,0.109061,-16.47240,107859281,1,0,100,0,26.570218,w,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663568,105169,20190628,GE1 200117P35000,1,20200117,20190212.0,P,35000,23.95,24.25,0,0,NaN,NaN,NaN,NaN,NaN,117999758,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
663569,105169,20190628,GE1 200117P4000,1,20200117,20190401.0,P,4000,0.00,0.05,0,236,NaN,NaN,NaN,NaN,NaN,127515131,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
663570,105169,20190628,GE1 200117P40000,1,20200117,20190228.0,P,40000,28.95,29.25,0,1,NaN,NaN,NaN,NaN,NaN,117999759,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0
663571,105169,20190628,GE1 200117P5000,1,20200117,20190626.0,P,5000,0.03,0.04,0,65745,NaN,NaN,NaN,NaN,NaN,117999760,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0


In [ ]:
purchsing_dates = 

In [41]:
opts['date'] = opts['date'].apply(str)

In [47]:
datetime.strptime(opts.iloc[0]['date'],"%Y%m%d")

datetime.datetime(2015, 6, 30, 0, 0)

In [93]:
calculate_arb(filtered.iloc[-1])

0.01 0.25 False 10.32 8.0


0

In [88]:
filtered

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume_x,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag,DATE,exOrEffDate,type,amount,declarationDate,recordDate,paymentDate,Unnamed: 6,timestamp,open,high,low,close,volume_y,millis,timestamp.1,time_of_last_tick,ARB
39186,105169,20150916,GE 150918C15000,1,20150918,20150916.0,C,15000,10.55,11.00,420,219,NaN,NaN,NaN,NaN,NaN,106240072,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,0.00
39187,105169,20150916,GE 150918C16000,1,20150918,NaN,C,16000,8.45,10.20,0,0,NaN,NaN,NaN,NaN,NaN,107284360,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,0.00
39188,105169,20150916,GE 150918C17000,1,20150918,20150617.0,C,17000,7.80,9.15,0,0,NaN,NaN,NaN,NaN,NaN,107284361,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,0.00
39189,105169,20150916,GE 150918C17500,1,20150918,NaN,C,17500,7.10,8.60,0,0,NaN,NaN,NaN,NaN,NaN,108806256,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,0.00
39190,105169,20150916,GE 150918C18000,1,20150918,20150916.0,C,18000,7.60,8.05,420,189,NaN,NaN,NaN,NaN,NaN,106240073,1,0,100,0,25.700235,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2015-09-16,09/17/2015,Cash,$0.23,09/11/2015,09/21/2015,10/26/2015,NaN,2015-09-16,25.31,26.03,25.31,25.31,57602745.0,1.442397e+09,2015-09-16,2015-09-16,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662952,105169,20190627,GE1 200117P35000,1,20200117,20190212.0,P,35000,24.10,24.30,0,0,NaN,NaN,NaN,NaN,NaN,117999758,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2019,Cash,$0.01,06/21/2019,07/01/2019,07/25/2019,NaN,2019-06-27,10.32,10.44,10.22,10.32,28629926.0,1.561627e+09,2019-06-27,2019-06-27,-315.39
662953,105169,20190627,GE1 200117P4000,1,20200117,20190401.0,P,4000,0.00,0.05,0,236,NaN,NaN,NaN,NaN,NaN,127515131,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2019,Cash,$0.01,06/21/2019,07/01/2019,07/25/2019,NaN,2019-06-27,10.32,10.44,10.22,10.32,28629926.0,1.561627e+09,2019-06-27,2019-06-27,-29.64
662954,105169,20190627,GE1 200117P40000,1,20200117,20190228.0,P,40000,29.10,29.35,0,1,NaN,NaN,NaN,NaN,NaN,117999759,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2019,Cash,$0.01,06/21/2019,07/01/2019,07/25/2019,NaN,2019-06-27,10.32,10.44,10.22,10.32,28629926.0,1.561627e+09,2019-06-27,2019-06-27,-360.34
662955,105169,20190627,GE1 200117P5000,1,20200117,20190626.0,P,5000,0.03,0.05,0,65745,NaN,NaN,NaN,NaN,NaN,117999760,1,0,100,1,-99.990000,NaN,NaN,NaN,36960410,GE,3600,0,1,NaN,0,622,GENERAL ELECTRIC CO.,NaN,A,0,2019-06-27,06/28/2019,Cash,$0.01,06/21/2019,07/01/2019,07/25/2019,NaN,2019-06-27,10.32,10.44,10.22,10.32,28629926.0,1.561627e+09,2019-06-27,2

In [ ]:
.01-.25 10.32-8 